# Analysis of rates

By combining elements from graph theory, convex optimization, and linear algebra, we can have powerful results on the stability of a stochastic system and its set of feasible rates.

`stochastic matching` provides the following information (cf https://hal.archives-ouvertes.fr/hal-03502084 for details):

- Graph type: injective-only, surjective-only, bijective, ``nonjective'';
- Kernel structure (for node and edge kernels);
- For given arrival rates: 
 - stability of the matching model;
 - Maximin and Moore-Penrose solutions of the conservation equation;
 - if relevant, a complete description of the vertices of $\Lambda_{\geqslant 0}$.



The purpose of this notebook is to give a tour of these analysis on some examples.

In [1]:
import stochastic_matching as sm
from stochastic_matching.display import VIS_OPTIONS
VIS_OPTIONS['height'] = 200

In [2]:
VIS_OPTIONS

{'interaction': {'navigationButtons': True}, 'width': '600px', 'height': 200}

## Tadpole

### Bijective tadpole

Consider a tadpole $T_{5, 3}$ with uniform arrival rates. Side note: if no rate is specified, degree-proportional rates are assumed, which ensure that the model is stabilizable if the graph is surjective.

In [3]:
tadpole = sm.Tadpole(m=5, n=3, rates='uniform')
tadpole.show_graph()

Its kernels are trivial: the graph is bijective.

In [4]:
tadpole.kernel

Kernels of a graph with 8 nodes and 8 edges.
Node dimension is 0.
Edge dimension is 0
Type: Bijective
Node kernel:
[]
Edge kernel:
[]

Let see the unique solution of the conservation equation:

In [5]:
tadpole.show_flow()

This does not sound very stable...

In [6]:
tadpole.stabilizable

False

Let's change the arrival rates to have something nicer.

In [7]:
tadpole.rates = [1, 1, 1, 1, 2, 2, 2, 1]
tadpole.show_flow()

Sounds better...

In [8]:
tadpole.stabilizable

True

### Nonjective tadpole

We now consider the bipartite tadpole $T_{4, 1}$, again with uniform arrival rates.

In [9]:
tadpole = sm.Tadpole(m=4, n=1, rates='uniform')
tadpole.show_graph()

Look at the kernel.

In [10]:
tadpole.kernel

Kernels of a graph with 5 nodes and 5 edges.
Node dimension is 1.
Edge dimension is 1
Type: Nonjective
Node kernel:
[[ 1]
 [-1]
 [ 1]
 [-1]
 [ 1]]
Edge kernel:
[[ 1 -1 -1  1  0]]

The node kernel shows the bipartite repartition of the nodes (*positive* and *negative* nodes).

The edge kernel shows how the rate flow can *slide* between edges. It can be displayed:

In [11]:
tadpole.show_kernel()

Note the black edge above: it indicates that the specific edge is unaffected by the kernel.

Let see the (attempt) solution of the conservation equation:

In [12]:
tadpole.show_flow()

The red nodes indicate that the conservation equation is not satisfied. This is not surprising, as the total arrival rates on positive nodes, 3, is distinct from the total arrival rates on negative nodes, 2.

We can change the rates in order to equalize the arrival rates.

In [13]:
tadpole.rates = [1, 1, 1, 2, 1]
tadpole.show_flow()

Note that the respect of the conservation law is not sufficient for stability: the fact that the graph is not surjective forbids stability.

In [14]:
tadpole.stabilizable

False

This does not forbid considering the vertices of the positive solutions.

In [15]:
tadpole.vertices

[{'kernel_coordinates': array([-0.5]),
  'edge_coordinates': array([0., 1., 1., 0., 1.]),
  'null_edges': [0, 3],
  'bijective': False},
 {'kernel_coordinates': array([0.5]),
  'edge_coordinates': array([1., 0., 0., 1., 1.]),
  'null_edges': [1, 2],
  'bijective': False}]

In [16]:
for i in range(len(tadpole.vertices)):
    tadpole.show_vertex(i, disp_rates=False)

## The diamond graph

Consider the following diamond:

In [17]:
diamond = sm.CycleChain(rates='uniform')
diamond.show_graph()

The graph structure is as follows:

In [18]:
diamond.kernel

Kernels of a graph with 4 nodes and 5 edges.
Node dimension is 0.
Edge dimension is 1
Type: Surjective-only
Node kernel:
[]
Edge kernel:
[[ 1 -1  0 -1  1]]

We can look at the kernel structure.

In [19]:
diamond.show_kernel(disp_flow=True)

We observe a null edge. As it is black (outside the edge kernel), it will remain null in all solutions of the conservation equation. In other words, the model is not stabilizable.

In [20]:
diamond.stabilizable

False

Let make it stabilizable.

In [21]:
diamond.rates = [2, 3, 2, 1]

In [22]:
diamond.show_kernel(disp_flow=True)

Better! Note that the represented base flow is the Moore-Penrose inverse. We can use the maximin flow if we prefer.

In [23]:
diamond.show_kernel(disp_flow=True, flow=diamond.maximin)

Another feature: looking the incompressible flow, which is the minimal rate achieved in any positive solution.

In [24]:
diamond.show_flow(flow=diamond.incompressible_flow(), check_flow=False, disp_zero=False)

To finish, let have a look at the vertices.

In [25]:
diamond.vertices

[{'kernel_coordinates': array([-0.25]),
  'edge_coordinates': array([1., 1., 1., 1., 0.]),
  'null_edges': [4],
  'bijective': True},
 {'kernel_coordinates': array([0.75]),
  'edge_coordinates': array([2., 0., 1., 0., 1.]),
  'null_edges': [1, 3],
  'bijective': False}]

In [26]:
for i in range(len(tadpole.vertices)):
    diamond.show_vertex(i)

## Not connected simple graphs

The package deals with simple graphs even if they are not connected. Consider a graph made of a square, a complete graph of size 4, a diamond, a paw and a three-edged star:

In [27]:
VIS_OPTIONS['height'] = 600
sample = sm.concatenate([sm.Cycle(4), sm.Complete(4), sm.CycleChain(), sm.Tadpole(), sm.Star()], 0 )
sample.show_graph()

The kernel will display global information:

In [28]:
sample.kernel

Kernels of a graph with 20 nodes and 22 edges.
Node dimension is 2.
Edge dimension is 4
Type: Nonjective
Node kernel:
[[ 0  1]
 [ 0 -1]
 [ 0  1]
 [ 0 -1]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [ 0  0]
 [-1  0]
 [ 1  0]
 [ 1  0]
 [ 1  0]]
Edge kernel:
[[ 1 -1 -1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1 -1  0  0 -1  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 -1  1  1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1 -1  0 -1  1  0  0  0  0  0  0  0]]

- The two node vectors point to the bipartite connected components (the square and the star).
- The four edge vectors point to the even cycles from the square, complete graph, and diamond.

If we want further information, we can look at the `connected_components` attribute.

In [29]:
sample.connected_components

[{'nodes': {0, 1, 2, 3},
  'edges': {0, 1, 2, 3},
  'spanner': {0, 1, 2},
  'pivot': False,
  'seeds': {3},
  'type': 'Bipartite with cycle(s)'},
 {'nodes': {4, 5, 6, 7},
  'edges': {4, 5, 6, 7, 8, 9},
  'spanner': {4, 5, 6},
  'pivot': 8,
  'seeds': {7, 9},
  'type': 'Non-bipartite polycyclic'},
 {'nodes': {8, 9, 10, 11},
  'edges': {10, 11, 12, 13, 14},
  'spanner': {10, 11, 13},
  'pivot': 12,
  'seeds': {14},
  'type': 'Non-bipartite polycyclic'},
 {'nodes': {12, 13, 14, 15},
  'edges': {15, 16, 17, 18},
  'spanner': {15, 16, 18},
  'pivot': 17,
  'seeds': set(),
  'type': 'Non-bipartite monocyclic'},
 {'nodes': {16, 17, 18, 19},
  'edges': {19, 20, 21},
  'spanner': {19, 20, 21},
  'pivot': False,
  'seeds': set(),
  'type': 'Tree'}]

Connected components give the following information for each connected component:
- The nodes and edges that belong to that component (or more accurately their index);
- A spanner of the component, e.g. a set of edges that form a spanning tree for the component;
- If the component is surjective, a *pivot*, e.g. an edge that makes the spanner non-bipartite if we add it;
- *Seeds*, which are a set of edges that can be used to build a edge kernel basis made of cycles and kayak paddles.
- The type of the component. For example, for a connected component, injective-only is a synonym for tree.

All operations that were previously performed can be done on graphs that are not connected.

In [30]:
sample.show_kernel(disp_flow=True)

## Hypergraphs

### Candy

The candy is a simple example of true hypergraph.

In [31]:
candy = sm.HyperPaddle()
candy.show_graph()

It is a bijective graph.

In [32]:
candy.kernel

Kernels of a graph with 7 nodes and 7 edges.
Node dimension is 0.
Edge dimension is 0
Type: Bijective
Node kernel:
[]
Edge kernel:
[]

With degree-proportional arrival rates, it is stabilizable.

In [33]:
candy.stabilizable

True

With uniform arrival rates, it is not stabilizable.

In [34]:
candy.rates = 'uniform'
candy.stabilizable

False

In [35]:
candy.show_flow()

### Fans

Fans yield nice examples of hypergraphs with cool kernels. The basic fan is called a *clover*.

In [36]:
clover = sm.Fan()
clover.show_graph()

In [37]:
clover.kernel

Kernels of a graph with 9 nodes and 10 edges.
Node dimension is 0.
Edge dimension is 1
Type: Surjective-only
Node kernel:
[]
Edge kernel:
[[-0.2773501 -0.2773501  0.2773501 -0.2773501 -0.2773501  0.2773501
  -0.2773501 -0.2773501  0.2773501  0.5547002]]

In [38]:
clover.show_kernel()

The double fan expands the kernel dimension.

In [39]:
double = sm.Fan(hyperedges=2)
double.show_graph()

In [40]:
double.kernel

Kernels of a graph with 9 nodes and 11 edges.
Node dimension is 0.
Edge dimension is 2
Type: Surjective-only
Node kernel:
[]
Edge kernel:
[[-0.39252987 -0.1694113   0.1694113  -0.39252987 -0.1694113   0.1694113
  -0.39252987 -0.1694113   0.1694113   0.56194117  0.22311857]
 [-0.21429023  0.31032211 -0.31032211 -0.21429023  0.31032211 -0.31032211
  -0.21429023  0.31032211 -0.31032211 -0.09603188  0.52461234]]

In [41]:
double.show_kernel()

Last but not least, the *bipartite* fan created a large node kernel.

In [42]:
bip = sm.Fan(cycle_size=4)
bip.show_graph()

In [43]:
bip.kernel

Kernels of a graph with 12 nodes and 13 edges.
Node dimension is 2.
Edge dimension is 3
Type: Nonjective
Node kernel:
[[ 0.24565655  0.32606675]
 [-0.24565655 -0.32606675]
 [ 0.24565655  0.32606675]
 [-0.24565655 -0.32606675]
 [-0.40521037  0.04971143]
 [ 0.40521037 -0.04971143]
 [-0.40521037  0.04971143]
 [ 0.40521037 -0.04971143]
 [ 0.15955382 -0.37577819]
 [-0.15955382  0.37577819]
 [ 0.15955382 -0.37577819]
 [-0.15955382  0.37577819]]
Edge kernel:
[[ 0.45677855 -0.45677855 -0.45677855  0.45677855 -0.0741917   0.0741917
   0.0741917  -0.0741917  -0.18933819  0.18933819  0.18933819 -0.18933819
   0.        ]
 [ 0.20305383 -0.20305383 -0.20305383  0.20305383  0.19174096 -0.19174096
  -0.19174096  0.19174096  0.41473431 -0.41473431 -0.41473431  0.41473431
   0.        ]
 [ 0.01106809 -0.01106809 -0.01106809  0.01106809 -0.45577516  0.45577516
   0.45577516 -0.45577516  0.20529613 -0.20529613 -0.20529613  0.20529613
   0.        ]]

The node kernel can be interpreted as follows:
- There are three *bipartite* components, one for each square. Along the square, the nodes value must alternate positively and negatively.
- The central edge forces the sum of the weights of its adjacent nodes to be zero. so if we fix two squares, the values on the last square are fully determined.

In [44]:
bip.show_kernel()

Remark that in this example, the hyperedge is not part of the edge kernel. Only the three regular squares are.

Just for the experience, let us add another hyperedge.

In [45]:
bip = sm.Fan(cycle_size=4, hyperedges=2)
bip.show_graph()

In [46]:
bip.kernel

Kernels of a graph with 12 nodes and 14 edges.
Node dimension is 2.
Edge dimension is 4
Type: Nonjective
Node kernel:
[[ 0.02781254 -0.4072998 ]
 [-0.02781254  0.4072998 ]
 [ 0.02781254 -0.4072998 ]
 [-0.02781254  0.4072998 ]
 [-0.36663824  0.17956354]
 [ 0.36663824 -0.17956354]
 [-0.36663824  0.17956354]
 [ 0.36663824 -0.17956354]
 [ 0.33882571  0.22773626]
 [-0.33882571 -0.22773626]
 [ 0.33882571  0.22773626]
 [-0.33882571 -0.22773626]]
Edge kernel:
[[ 0.02607132  0.16774262  0.16774262 -0.16774262  0.5563414  -0.36252746
  -0.36252746  0.36252746  0.3095347  -0.11572076 -0.11572076  0.11572076
  -0.19381393 -0.19381393]
 [ 0.35172883 -0.44936338 -0.44936338  0.44936338  0.1434273  -0.24106185
  -0.24106185  0.24106185 -0.18434102  0.08670647  0.08670647 -0.08670647
   0.09763455  0.09763455]
 [-0.44421226  0.16512197  0.16512197 -0.16512197 -0.02581079 -0.2532795
  -0.2532795   0.2532795  -0.48930015  0.21020986  0.21020986 -0.21020986
   0.27909029  0.27909029]
 [-0.24624282 -0.085

In [47]:
bip.show_kernel()